# Preparation

## Import

In [ ]:
# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow import keras
from tensorflow.keras import layers
import PIL
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


**AUTOTUNE**: 

* **map**: transform the dataset according to a function, in this case converts the images to float
* **cache**: keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.
* **prefetch**: for optimization porpuses 

## Functions

In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

### create dataset

In [ ]:
def create_train_dataset(training_dir, batch_size = 32, greyscale = True):

    color = 'grayscale'
    if(not greyscale):
        color = 'rgb'

    # for fitting 
    train_ds = tf.keras.utils.image_dataset_from_directory(
        training_dir,
        seed=420,
        image_size=(img_height, img_width),
        batch_size= batch_size,
        validation_split=0.2,
        subset="training",
        color_mode=color,
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        training_dir,
        seed=420,
        image_size=(img_height, img_width),
        batch_size= batch_size,
        validation_split=0.2,
        subset="validation",
        color_mode= color,
    )




    AUTOTUNE = tf.data.experimental.AUTOTUNE

    ds_train = train_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)
    ds_valid = val_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)
   
    class_names = train_ds.class_names

    

    return (ds_train, ds_valid, class_names)



def create_test_dataset( query_ds, gallery_ds,  batch_size = 32, greyscale = True):
    color = 'grayscale'
    if(not greyscale):
        color = 'rgb'   

    query_ds = tf.keras.utils.image_dataset_from_directory(
        query_dir,
        seed=420,
        image_size=(img_height, img_width),
        batch_size= batch_size,
        color_mode= color,
    )

    gallery_ds = tf.keras.utils.image_dataset_from_directory(
        gallery_dir,
        seed=420,
        image_size=(img_height, img_width),
        batch_size= batch_size,
        color_mode= color,
    )

    AUTOTUNE = tf.data.experimental.AUTOTUNE

    ds_query = query_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)
    ds_gallery = gallery_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

    return (ds_query, ds_gallery)

### print 16 images

In [ ]:
def print_16_imgs(dataset):
    imgs = np.concatenate([x for x, y in dataset], axis=0)
    labels = np.concatenate([y for x, y in dataset], axis=0)

    plt.figure(figsize=(10,10))
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(imgs[i].astype('uint8'), cmap=plt.cm.binary)
        plt.grid(False)
        plt.title(class_names[labels[i]])



### create model

In [ ]:
def create_model():

    data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal", input_shape=(img_height, img_width, img_depth)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
    )


    model = keras.Sequential([

        data_augmentation,
        layers.Rescaling(1./255), # only with rgb

        layers.Conv2D(32, 3, padding='same', activation='relu' ),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, padding='same', activation='relu'), # relu for hidden layers
        layers.MaxPooling2D(),

        layers.Dropout(0.3),
        layers.Flatten(),
        #layers.Dense(16),
        layers.Dense(32, activation='relu'),
       # layers.Dense(10, activation='softmax') # softmax for probablities 


    ])

    model.compile(
        optimizer='adam',
        loss= tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'],
    )

    model.build((None, img_width, img_height, img_depth))


    model.summary()
    return model



## Parameters

In [ ]:
img_height = 128
img_width = 128
img_depth = 3 # 1 if greyscale 3 if rgb
n_epochs = 10

# Dataset and model

## Dataset directories

In [ ]:
# drive 
training_dir = pathlib.Path('/datasets/animali/animals_dataset_the_ostriches/animals_dataset/training')
query_dir = pathlib.Path('/datasets/animali/animals_dataset_the_ostriches/animals_dataset/validation2/query')
gallery_dir = pathlib.Path('/datasets/animali/animals_dataset_the_ostriches/animals_dataset/validation2/gallery')
#local
#training_dir = pathlib.Path('/work/dataset/training')
#query_dir = pathlib.Path('/work/dataset/validation/query')
#gallery_dir = pathlib.Path('/work/dataset/validation/gallery')



## Create dataset and fit the model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        training_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        #class_mode='binary'
        )


model = create_model()

model.fit(train_generator,
        steps_per_epoch=2000,
        epochs=50,
       # validation_data=validation_generator,
        validation_steps=800)



Found 16607 images belonging to 10 classes.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 128, 128, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 64)       0         
 2D)      

InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/usr/local/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/usr/local/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/shared-libs/python3.7/py-core/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
      app.start()
    File "/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 619, in start
      self.io_loop.start()
    File "/shared-libs/python3.7/py-core/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever(

# Evaluation and prediction

## Functions

### Get rank 

In [ ]:
# given a prediction sorts according to the probabilities and returns a list of tuples (1, 0.2)
# the tuple (1, 0.2) means that class 1 has 0.2 % of probability
def get_sorted_prediction(predictions):
    a = np.argsort(predictions)[::-1] # sorted indexes 
    b = sorted(predictions)[::-1] # sorted probabilities 
    return zip(a,b)

# this func is called for each query image return 10 element of the gallery (by their index)
def get_rank(query_pred, gallery_pred):

    # sort the 
    query = list(get_sorted_prediction(query_pred)) # query

    #gallery 
    gallery = {}
    for i, pred in enumerate(gallery_pred):
        gallery[i] = (list(get_sorted_prediction(pred)))

    ranking = []

    # iterate starting from the most probable class and find a match in the gallery 
    for class_number in query:
        maxi = 0
        max_idx = 99
        for i, pred in gallery.items():
            if class_number[0] == pred[0][0]:
                if maxi < pred[0][1]:
                    maxi = pred[0][1]
                    max_idx = i
        

        if maxi > 0:
            ranking.append(max_idx)

    return ranking



## Evaluate

## Ranking

# PARTO DA QUI

In [ ]:
class Dataset(object):
    def __init__(self, data_path):
        #path of the dataset
        self.data_path = data_path

        #class list
        self.data_classes = [directory for directory in os.listdir(data_path) if os.path.isdir(data_path+directory)]

        # init lists and dictionary
        self.images = []
        self.labels = []
        self.class_names = {}

        # for each class and for each image save the image and the label in the lists 
        for c, c_name in enumerate(self.data_classes):
            temp_path = os.path.join(self.data_path, c_name)
            temp_images = os.listdir(temp_path)
            self.class_names[c] = c_name

            for i in temp_images:
                img_tmp = os.path.join(temp_path, i)


                if img_tmp.endswith('.jpg') or img_tmp.endswith('.JPEG'):
                   # img = image.load_img(img_tmp, target_size=(224,224))
                    img = cv2.imread(img_tmp, 3)
                    model_image_size = (128, 128)
                    resized_image = cv2.resize(img, model_image_size, interpolation = cv2.INTER_CUBIC)
                    resized_image = resized_image.astype(np.float32) / 255.0
                    self.images.append(resized_image)
                    self.labels.append(c)
                    

        print('Loaded {:d} images from {:s} '.format(len(self.images), self.data_path))



    def num_classes(self):
        # returns number of classes of the dataset
        return len(self.data_classes)
    
    def get_dataset(self):
        return (list(zip(self.images, self.labels)), self.class_names)

    def generate(self):

        datagen = ImageDataGenerator(
            rotation_range=10, # rotation
            width_shift_range=0.2, # horizontal shift
            height_shift_range=0.2, # vertical shift
            zoom_range=0.2, # zoom
            horizontal_flip=True, # horizontal flip
            brightness_range=[0.2,1.2]
            ) # brightness

        train_generator = datagen.flow_from_directory(
                  directory=self.data_path,
                  target_size=(128, 128), # resize to this size
                  color_mode="rgb", # for coloured images
                  batch_size=32, # number of images to extract from folder for every batch
                  #class_mode="binary", # classes to predict
                  seed=420 # to make the result reproducible
                  )


In [ ]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import cv2 


# Load the pretrained model
model = models.resnet18(pretrained=True)


# Use the model object to select the desired layer
layer = model._modules.get('avgpool')


# Set model to evaluation mode
model.eval()


scaler = transforms.Resize(224)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()





AttributeError: 'ResNet' object has no attribute 'summary'

In [ ]:
pic_one_vector = get_vector('/work/dataset/training/n01443537(goldfish)/n01443537_483.JPEG')
pic_two_vector = get_vector('/work/dataset/training/n01443537(goldfish)/n01443537_640.JPEG')


# Using PyTorch Cosine Similarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(pic_one_vector.unsqueeze(0),
              pic_two_vector.unsqueeze(0))
print('\nCosine similarity: {0}\n'.format(cos_sim))

RuntimeError: output with shape [512] doesn't match the broadcast shape [1, 512, 1, 512]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=93aceac2-8452-469e-8b02-c16d0438aa9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>